In [ ]:
#kaggle dataset-

In [50]:
import pandas as pd
import sys
# Why does it take so much time to import cudf?
import cudf
import numpy as np

# Loading the transaction data-

In [51]:
%%time
df = pd.read_csv('/raid/manass/cudf/data/half_transactions.csv')
sys.getsizeof(df)/10**9

CPU times: user 4min 33s, sys: 24.3 s, total: 4min 58s
Wall time: 4min 51s


58.897970187

In [52]:
df.shape

(130984859, 13)

In [53]:
df.head()

,Unnamed: 0,CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,CUST_ID_len,TRANS_ID_len
0,0,CI6XLYUMQK,2015-05-01,NaN,T8I9ZB5A6X90UG8,2015-09-11,2015,9,11,Motor/Travel,20.27,10,15
1,1,CI6XLYUMQK,2015-05-01,NaN,TZ4JSLS7SC7FO9H,2017-02-08,2017,2,8,Motor/Travel,12.85,10,15
2,2,CI6XLYUMQK,2015-05-01,NaN,TTUKRDDJ6B6F42H,2015-08-01,2015,8,1,Housing,383.80,10,15
3,3,CI6XLYUMQK,2015-05-01,NaN,TDUHFRUKGPPI6HD,2019-03-16,2019,3,16,Entertainment,5.72,10,15
4,4,CI6XLYUMQK,2015-05-01,NaN,T0JBZHBMSVRFMMD,2015-05-15,2015,5,15,Entertainment,11.06,10,15


# Create a customer dataset with age and income level-

In [54]:
%%time
df_customer = pd.DataFrame()
df_customer['CUST_ID'] = df['CUST_ID'].unique()

CPU times: user 4.52 s, sys: 0 ns, total: 4.52 s
Wall time: 4.41 s


In [55]:
%%time
income_levels = ['Less than 10K', 'Between 10K and 25K', 'Greater than 25K', 'Between 25K and 100K','Greater than 100K']
df_customer['age'] = np.random.randint(18, 81, size=len(df_customer))
df_customer['Income Level'] = np.random.choice(income_levels, size=len(df_customer))

CPU times: user 8.16 ms, sys: 0 ns, total: 8.16 ms
Wall time: 6.94 ms


In [56]:
df_customer.shape, df_customer.columns

((35921, 3), Index(['CUST_ID', 'age', 'Income Level'], dtype='object'))

# Merge df_customer with df

In [57]:
%%time
df_cust_transactions = pd.merge(df,df_customer, on = 'CUST_ID', how = 'left')

CPU times: user 17 s, sys: 0 ns, total: 17 s
Wall time: 16.6 s


In [58]:
df_cust_transactions.head()

,Unnamed: 0,CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,CUST_ID_len,TRANS_ID_len,age,Income Level
0,0,CI6XLYUMQK,2015-05-01,NaN,T8I9ZB5A6X90UG8,2015-09-11,2015,9,11,Motor/Travel,20.27,10,15,40,Greater than 100K
1,1,CI6XLYUMQK,2015-05-01,NaN,TZ4JSLS7SC7FO9H,2017-02-08,2017,2,8,Motor/Travel,12.85,10,15,40,Greater than 100K
2,2,CI6XLYUMQK,2015-05-01,NaN,TTUKRDDJ6B6F42H,2015-08-01,2015,8,1,Housing,383.80,10,15,40,Greater than 100K
3,3,CI6XLYUMQK,2015-05-01,NaN,TDUHFRUKGPPI6HD,2019-03-16,2019,3,16,Entertainment,5.72,10,15,40,Greater than 100K
4,4,CI6XLYUMQK,2015-05-01,NaN,T0JBZHBMSVRFMMD,2015-05-15,2015,5,15,Entertainment,11.06,10,15,40,Greater than 100K


In [59]:
# Remove all the non-numeric values from AMOUNT column-

# INSIGHT: Each change costs so much time and mental energy if the processing is slow-

In [60]:
%%time
df_cust_transactions = df_cust_transactions[pd.to_numeric(df_cust_transactions['AMOUNT'], errors='coerce').notna()]

CPU times: user 17.3 s, sys: 10.6 s, total: 27.9 s
Wall time: 27.3 s


# Time to filter-

In [61]:
%%time
df_cust_transactions[df_cust_transactions['EXP_TYPE'] == 'Groceries']

CPU times: user 9.56 s, sys: 0 ns, total: 9.56 s
Wall time: 9.3 s


,Unnamed: 0,CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,CUST_ID_len,TRANS_ID_len,age,Income Level
8,8,CI6XLYUMQK,2015-05-01,NaN,TQJ3BTAB7E449ZN,2018-12-23,2018,12,23,Groceries,26.77,10,15,40,Greater than 100K
10,10,CI6XLYUMQK,2015-05-01,NaN,TEDY6MR01ELTHWY,2018-02-25,2018,2,25,Groceries,9.27,10,15,40,Greater than 100K
11,11,CI6XLYUMQK,2015-05-01,NaN,TRUQRD6ME4A13I0,2019-04-15,2019,4,15,Groceries,8.97,10,15,40,Greater than 100K
13,13,CI6XLYUMQK,2015-05-01,NaN,T8DBBZQXIR0X93O,2015-06-14,2015,6,14,Groceries,8.24,10,15,40,Greater than 100K
15,15,CI6XLYUMQK,2015-05-01,NaN,T0TGOW0T3504WJ3,2020-05-06,2020,5,6,Groceries,17.44,10,15,40,Greater than 100K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130984839,130984839,C80JX019HW,2016-12-01,NaN,TQ32ZJAAO2MDS42,2020-05-06,2020,5,6,Groceries,147.16,10,15,24,Greater than 25K
130984848,130984848,C80JX019HW,2016-12-01,NaN,THBH6J8YE1SDC4I,2019-02-04,2019,2,4,Groceries,197.36,10,15,24,Greater than 25K
130984851,130984851,C80JX019HW,2016-12-01,NaN,THGL384VF12IKED,2020-04-27,2020,4,27,Groceries,177.02,10,15,24,Greater than 25K
130984853,130984853,C80JX019HW,2016-12-01,NaN,TUZY62U9QF6AP3U,2019-04-24,2019,4,24,Groceries,130.32,10,15,24,Greater than 25K


In [62]:
# For each age and income level what is the mean expenditure per customer per transaction

# INSIGHT: Debugging takes a longer time- (1 minute wait before I knew I there was a string entry in the age column) -

In [63]:
df_cust_transactions.columns

Index(['Unnamed: 0', 'CUST_ID', 'START_DATE', 'END_DATE', 'TRANS_ID', 'DATE',
       'YEAR', 'MONTH', 'DAY', 'EXP_TYPE', 'AMOUNT', 'CUST_ID_len',
       'TRANS_ID_len', 'age', 'Income Level'],
      dtype='object')

In [64]:
%%time
df_cust_transactions.groupby(['EXP_TYPE',
    'age', 'Income Level']).mean(['AMOUNT'])

CPU times: user 19.3 s, sys: 6.23 s, total: 25.5 s
Wall time: 25 s


Unnamed: 0         YEAR  \
EXP_TYPE            age Income Level                                      
Bills and Utilities 18  Between 10K and 25K   6.567805e+07  2016.274627   
                        Between 25K and 100K  6.234058e+07  2016.516408   
                        Greater than 100K     6.947977e+07  2016.198667   
                        Greater than 25K      6.165267e+07  2016.462734   
                        Less than 10K         6.654085e+07  2016.157385   
...                                                    ...          ...   
Tax                 80  Between 10K and 25K   6.725035e+07  2016.061571   
                        Between 25K and 100K  6.120094e+07  2016.609736   
                        Greater than 100K     6.754168e+07  2016.030068   
                        Greater than 25K      6.704178e+07  2016.133015   
                        Less than 10K         6.237858e+07  2015.804639   

                                                 MONTH        DAY      AMOUNT  \
EXP_TYPE            age Income Level                                            
Bills and Utilities 18  Between 10K and 25K   6.619248  10.879051  267.081043   
                        Between 25K and 100K  6.618700  10.498194  268.372670   
                        Greater than 100K     6.592023  10.664600  279.192840   
                        Greater than 25K      6.635408  10.598253  252.665610   
                        Less than 10K         6.605620  10.469367  274.045225   
...                                                ...        ...         ...   
Tax                 80  Between 10K and 25K   6.598726   5.053432  583.445858   
                        Between 25K and 100K  6.623584   5.553504  541.335323   
                        Greater than 100K     6.574617   5.513374  621.696903   
                        Greater than 25K      6.605442   5.742220  626.225735   
                        Less than 10K         6.569845   5.299227  601.830402   

                                              CUST_ID_len  TRANS_ID_len  
EXP_TYPE            age Income Level                                     
Bills and Utilities 18  Between 10K and 25K          10.0          15.0  
                        Between 25K and 100K         10.0          15.0  
                        Greater than 100K            10.0          15.0  
                        Greater than 25K             10.0          15.0  
                        Less than 10K                10.0          15.0  
...                                                   ...           ...  
Tax                 80  Between 10K and 25K          10.0          15.0  
                        Between 25K and 100K         10.0          15.0  
                        Greater than 100K            10.0          15.0  
                        Greater than 25K             10.0          15.0  
                        Less than 10K                10.0          15.0  

[3780 rows x 7 columns]

# Sorting-

# Customer with the biggest transaction-

# Insight - More time spent waiting for the kernel to reset and the output to be deallocated.

In [70]:
%%time
df_cust_transactions.groupby('CUST_ID')['AMOUNT'].sum().sort_values(ascending=False)

CPU times: user 26.9 s, sys: 466 ms, total: 27.3 s
Wall time: 26.8 s


130958849    6506.81
129672182    6506.68
130956925    6480.41
129674064    6480.28
127312723    6476.55
              ...   
128670658       1.30
130251629       1.29
130251023       1.29
130821858       1.29
128672891       1.28
Name: AMOUNT, Length: 130984859, dtype: float64

# Row wise operations

In [66]:
def calculate_tax(row):
    # Placeholder tax calculation
    return row['AMOUNT'] * (1.0-0.3) if row['EXP_TYPE'] == 'Income' else 0

In [67]:
%%time
df_cust_transactions['adjusted_income'] = df_cust_transactions.apply(calculate_tax, axis=1)

CPU times: user 8min 47s, sys: 18 s, total: 9min 5s
Wall time: 8min 54s


# Rolling average of entertainment related expense-

In [68]:
%%time
df_cust_transactions['rolling_average'] = df_cust_transactions[
df_cust_transactions['EXP_TYPE'
] == 'Entertainment'].groupby(
    'CUST_ID')[
'AMOUNT'].rolling(window=90).mean().reset_index(
    level=0, drop=True)

CPU times: user 1min 2s, sys: 5.36 s, total: 1min 7s
Wall time: 1min 6s


# Handling Missing Data

In [69]:
%%time

df_cust_transactions['AMOUNT'] = df_cust_transactions['AMOUNT'].interpolate(method='linear')

CPU times: user 382 ms, sys: 0 ns, total: 382 ms
Wall time: 372 ms


# Cudf-

# Sometimes I get OOM error and sometimes I get Large String Error

# 21 GB dataset

In [4]:
%%time
#df = pd.read_csv('/raid/manass/cudf/data/transactions.csv')
gdf = cudf.read_csv('/raid/manass/cudf/data/transactions.csv', sep=',')
sys.getsizeof(gdf)/10**9

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /raid/manass/miniconda3/envs/rapids-24.06/include/rmm/mr/device/cuda_memory_resource.hpp

# limit the size of df and then store it back-

In [147]:
top_50_percent_rows = len(df) // 2
top_50_df = df.head(top_50_percent_rows)
top_50_df.shape
top_50_df.to_csv('/raid/manass/cudf/data/half_transactions.csv')

# Loaded the half size dataset (13 GB) onto GPU.

# Everytime I load the dataframe -- a new copy of the data gets loaded to the dataframe

In [2]:
%%time
gdf = cudf.read_csv('/raid/manass/cudf/data/half_transactions.csv')

CPU times: user 6.28 s, sys: 3.94 s, total: 10.2 s
Wall time: 10.1 s


In [3]:
%%time
gdf.head()

CPU times: user 7.92 ms, sys: 6.94 ms, total: 14.9 ms
Wall time: 13.5 ms


,Unnamed: 0,CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,CUST_ID_len,TRANS_ID_len
0,0,CI6XLYUMQK,2015-05-01,<NA>,T8I9ZB5A6X90UG8,2015-09-11,2015,9,11,Motor/Travel,20.27,10,15
1,1,CI6XLYUMQK,2015-05-01,<NA>,TZ4JSLS7SC7FO9H,2017-02-08,2017,2,8,Motor/Travel,12.85,10,15
2,2,CI6XLYUMQK,2015-05-01,<NA>,TTUKRDDJ6B6F42H,2015-08-01,2015,8,1,Housing,383.80,10,15
3,3,CI6XLYUMQK,2015-05-01,<NA>,TDUHFRUKGPPI6HD,2019-03-16,2019,3,16,Entertainment,5.72,10,15
4,4,CI6XLYUMQK,2015-05-01,<NA>,T0JBZHBMSVRFMMD,2015-05-15,2015,5,15,Entertainment,11.06,10,15


In [4]:
%%time
df_customer = cudf.DataFrame()
df_customer['CUST_ID'] = gdf['CUST_ID'].unique()

CPU times: user 73.7 ms, sys: 12.9 ms, total: 86.6 ms
Wall time: 83.8 ms


In [5]:
%%time
income_levels = ['Less than 10K', 'Between 10K and 25K', 'Greater than 25K', 'Between 25K and 100K','Greater than 100K']
df_customer['age'] = np.random.randint(18, 81, size=len(df_customer))
df_customer['Income Level'] = np.random.choice(income_levels, size=len(df_customer))

CPU times: user 8.86 ms, sys: 4.62 ms, total: 13.5 ms
Wall time: 11.9 ms


In [6]:
# Calculate total memory usage of the DataFrame in bytes
total_memory_bytes = df_customer.memory_usage(deep=True).sum()

# Convert bytes to gigabytes
total_memory_gb = total_memory_bytes / (1024**3)

total_memory_gb

0.0014389343559741974

In [7]:
%%time
df_cust_transactions = cudf.merge(gdf,df_customer, on = 'CUST_ID', how = 'left')

OverflowError: CUDF failure at: /opt/conda/conda-bld/work/cpp/include/cudf/strings/detail/strings_children.cuh:85: Size of output exceeds the column size limit

In [8]:
gdf_small=gdf.head(len(gdf)//2)

In [9]:
# Calculate total memory usage of the DataFrame in bytes
total_memory_bytes = gdf_small.memory_usage(deep=True).sum()

# Convert bytes to gigabytes
total_memory_gb = total_memory_bytes / (1024**3)

total_memory_gb

8.695324851199985

In [10]:
%%time
gdf_cust_transactions = cudf.merge(gdf_small,df_customer, on = 'CUST_ID', how = 'left')

CPU times: user 120 ms, sys: 15.1 ms, total: 135 ms
Wall time: 131 ms


In [11]:
# Calculate total memory usage of the DataFrame in bytes
total_memory_bytes = gdf_small.memory_usage(deep=True).sum()

# Convert bytes to gigabytes
total_memory_gb = total_memory_bytes / (1024**3)

total_memory_gb

8.695324851199985

In [12]:
gdf.to_csv("/raid/manass/cudf/data/quarter_transactions.csv")

OverflowError: Writing CSV file with chunksize=130984859 failed. Consider providing a smaller chunksize argument.

In [13]:
gdf_small.to_csv("/raid/manass/cudf/data/quarter_transactions.csv")

OverflowError: Writing CSV file with chunksize=65492429 failed. Consider providing a smaller chunksize argument.

# Different processes with 8 GB data-

# Force to numeric-

In [14]:
%%time
gdf_cust_transactions = gdf_cust_transactions[cudf.to_numeric(gdf_cust_transactions['AMOUNT'], errors='coerce').notna()]

CPU times: user 17.1 ms, sys: 25.6 ms, total: 42.7 ms
Wall time: 38.5 ms


In [15]:
%%time
gdf_cust_transactions[gdf_cust_transactions['EXP_TYPE'] == 'Groceries']

CPU times: user 29.7 ms, sys: 24.4 ms, total: 54.1 ms
Wall time: 51.4 ms


,Unnamed: 0,CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,CUST_ID_len,TRANS_ID_len,age,Income Level
0,256,CI6XLYUMQK,2015-05-01,<NA>,T29K3BZ6FST1LJS,2017-06-01,2017,6,1,Groceries,7.89,10,15,69,Between 25K and 100K
18,5634,CF7T5CWU3D,2015-11-01,<NA>,TP27RDM0KB5APMU,2018-04-24,2018,4,24,Groceries,27.51,10,15,50,Greater than 100K
19,5635,CF7T5CWU3D,2015-11-01,<NA>,TXMGAJUQIEV9HQ1,2018-06-16,2018,6,16,Groceries,33.59,10,15,50,Greater than 100K
21,5637,CF7T5CWU3D,2015-11-01,<NA>,TJ9AAL7BD5SL6IM,2017-09-08,2017,9,8,Groceries,38.32,10,15,50,Greater than 100K
22,5638,CF7T5CWU3D,2015-11-01,<NA>,TGHX2EFFSO1S62V,2017-06-07,2017,6,7,Groceries,46.60,10,15,50,Greater than 100K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65492394,65486925,C25NX0HIW5,2013-02-01,2020-11-01,TY61I5RFKBHWB1K,2014-07-18,2014,7,18,Groceries,188.24,10,15,69,Between 10K and 25K
65492397,65486928,C25NX0HIW5,2013-02-01,2020-11-01,TWLJW4JPGXYWSH7,2015-02-19,2015,2,19,Groceries,184.55,10,15,69,Between 10K and 25K
65492416,65487909,CM9VR46CFD,2014-07-01,<NA>,TO85I82T72KPTIC,2019-08-10,2019,8,10,Groceries,48.16,10,15,72,Between 25K and 100K
65492418,65487911,CM9VR46CFD,2014-07-01,<NA>,TMEINW15FB1ZZBY,2014-07-24,2014,7,24,Groceries,59.75,10,15,72,Between 25K and 100K


# cudf and pandas API look different here-

# Stack overflow and other online sources don't cover all the APIs -- 
# More users, better issue resolution, resulting in more users onboarding

In [19]:
%%time
gdf_cust_transactions.groupby(['EXP_TYPE',
    'age', 'Income Level']).mean()['AMOUNT']

TypeError: function is not supported for this dtype: mean

In [21]:
# The below worked as APIs are different designed-- 

In [26]:
%%time
result = gdf_cust_transactions.groupby([
    'EXP_TYPE', 'age', 'Income Level'])[
'AMOUNT'].mean().reset_index()

CPU times: user 17.9 ms, sys: 7.27 ms, total: 25.2 ms
Wall time: 22.3 ms


In [24]:
%%time
gdf_cust_transactions.groupby('CUST_ID')[
'AMOUNT'].sum().sort_values(ascending=False)

CPU times: user 18.1 ms, sys: 2.3 ms, total: 20.4 ms
Wall time: 18.1 ms


CUST_ID
C6JC5K02HA    2178693.53
CR5558CHPJ    2092671.99
CA9UYOQ5DA    2028950.49
C6CWQR20K0    2016848.91
CDZKNLWO3Z    1901449.01
                 ...    
CCOX3VE7UC       9656.74
CBK5I605HN       9370.76
CIY6HO4JXR       9317.51
C29J7JQBQX       8974.90
CN2GN4FJ2Q       8112.56
Name: AMOUNT, Length: 17890, dtype: float64

In [43]:
def calculate_tax(row):
    # Placeholder tax calculation
    return row['AMOUNT'] * (1.0-0.3) if row['EXP_TYPE'] == 'Groceries' else 0

# Does not support rown wise operations (axis = 0)

In [44]:
%%time
gdf_cust_transactions['adjusted_income'] = gdf_cust_transactions.apply(calculate_tax, axis=1)

CPU times: user 27.6 ms, sys: 10.2 ms, total: 37.8 ms
Wall time: 34.9 ms


# Rolling average

Just had to change df_cust_transactions to gdf_cust_transactions and it worked!

In [45]:
%%time
gdf_cust_transactions['rolling_average'] = gdf_cust_transactions[
gdf_cust_transactions['EXP_TYPE'
] == 'Entertainment'].groupby(
    'CUST_ID')[
'AMOUNT'].rolling(window=90).mean().reset_index(
    level=0, drop=True)

CPU times: user 655 ms, sys: 0 ns, total: 655 ms
Wall time: 698 ms


In [46]:
gdf_cust_transactions['rolling_average']

0                  <NA>
1                  <NA>
2                  <NA>
3                  <NA>
4                  <NA>
               ...     
65492424    25.62777778
65492425           <NA>
65492426           <NA>
65492427           <NA>
65492428    71.70266667
Name: rolling_average, Length: 65492429, dtype: float64

# Missing Data interpolation

In [71]:
%%time

gdf_cust_transactions['AMOUNT'] = gdf_cust_transactions['AMOUNT'].interpolate(method='linear')

CPU times: user 14.7 ms, sys: 31.9 ms, total: 46.6 ms
Wall time: 43.7 ms
